In [ ]:
import NBInclude: @nbinclude
@nbinclude("./Data.ipynb");

In [ ]:
version = ""
dataset = ""
medium = ""
metric = "";

In [ ]:
function get_inputs(df, medium, metric)
    df = as_metric(df, metric)
    resid = df
    if metric == "rating"
        params = read_params("baseline/v1/$dataset/$medium/rating")
        user_biases = get_user_biases(resid, params)
        item_biases = params["a"]
        β = params["β"]
        Threads.@threads for i = 1:length(df.metric)
            df.metric[i] -=
                (get(user_biases, df.userid[i], 0) + item_biases[df.itemid[i]]) * β
        end
    end
    df
end

function model(dfs, userids, outdir)
    max_userid = max(maximum(userids), [maximum(x.userid) for x in values(dfs)]...)
    sparse(x, c, m) =
        SparseArrays.sparse(x.itemid, x.userid, getfield(x, c), num_items(m), max_userid)
    sparse_empty() = SparseArrays.sparse(Int32[], Int32[], Float32[], 1, max_userid)
    X = vcat(
        [
            sparse(get_inputs(dfs[m], m, metric), :metric, m) for
            metric in ["rating", "watch"] for m in ALL_MEDIUMS
        ]...,
    )
    Y = sparse_empty()
    W = sparse_empty()
    users = collect(userids)
    fn = "$outdir/predict.h5"
    save_features(X[:, users], Y[:, users], W[:, users], users, fn)

    args = [
        "python3",
        "pytorch.py",
        "--outdir",
        outdir,
        "--init",
        "$outdir/model.pt",
        "--predict",
        fn,
    ]
    run(`$args`)
    users, preds = HDF5.h5open("$fn.out", "r") do f
        g(x) = read(f[x])
        g("users"), g("predictions")
    end
    d = Dict(users[i] => preds[:, i] for i = 1:length(users))
    rm.([fn, "$fn.out"])
    d
end;

In [ ]:
const name = "bagofwords/$version/$dataset/$medium/$metric"
const outdir = get_data_path("alphas/$name")
write_alpha((df, userids) -> model(df, userids, outdir), medium, name)

In [ ]:
alphas = ["bagofwords/$version/$dataset/$medium/$metric"]
if metric == "rating"
    push!(alphas, "baseline/v1/$dataset/$medium/$metric")
end
print_losses(medium, metric, alphas);